In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('gemini_BTCUSD_2020_1min.csv')

In [3]:
data.head()

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1.618880e+12,4/20/2021 0:02,BTCUSD,55717.47,55723.00,55541.69,55541.69,2.406734
1,1.618880e+12,4/20/2021 0:01,BTCUSD,55768.94,55849.82,55711.74,55717.47,0.573458
2,1.618880e+12,4/20/2021 0:00,BTCUSD,55691.79,55793.15,55691.79,55768.94,3.309904
3,1.618880e+12,4/19/2021 23:59,BTCUSD,55777.86,55777.86,55677.92,55691.79,0.684934
4,1.618880e+12,4/19/2021 23:58,BTCUSD,55803.50,55823.88,55773.08,55777.86,0.477225


In [5]:
data = data.iloc[: , 3:]

In [10]:
data.shape

(658065, 5)

In [6]:
scaler = MinMaxScaler().fit(data)

In [7]:
Data = scaler.transform(data)

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [20]:
class Data_Gen(Dataset):
    def __init__(self, data, window, prediction, step,min_index, max_index):
        self.data = data
        self.window = window
        self.prediction = prediction
        self.step = step
        self.min_index = min_index
        self.max_index = max_index
    def __len__(self):
        return (self.max_index - self.min_index - self.window - self.prediction - 1)
    def __getitem__(self, index):
        start_index = index + self.min_index
        end_index = start_index + self.window
        indices = range(start_index , end_index, self.step)

        samples = np.zeros((len(indices), 4))

        targets  = np.zeros((1, ))
        for j , idx in enumerate(indices):
            samples[j] = self.data[idx, 1:]
        targets = self.data[idx + self.prediction, 0]
        samples = torch.from_numpy(samples).float()
        targets = torch.from_numpy(targets).float()

        

        return samples, targets

In [15]:
window = 1440
predcition = 10
step = 1
batch = 32


In [21]:
Train_dataset = Data_Gen(data = Data, window =window, prediction = predcition,step= step, min_index =200000, max_index=300000)
Val_dataset = Data_Gen(data = Data, window =window, prediction = predcition,step= step, min_index =300000, max_index=400000)
Test_dataset = Data_Gen(data = Data, window =window, prediction = predcition,step= step, min_index = 400001,max_index= 500000)

In [22]:
Train_loader = DataLoader(Train_dataset, batch_size = batch, shuffle = True)
Train_loader = DataLoader(Val_dataset, batch_size = batch, shuffle = True)
Train_loader = DataLoader(Test_dataset, batch_size = batch, shuffle = True)

In [23]:
class RNN1(nn.Module):
    def __init__(self):
        super(RNN1, self).__init__()
        self.gru1 = nn.GRU(4, 32, batch_first=True, bidirectional= False , num_layers = 1)
        self.gru2 = nn.GRU(32, 32, batch_first = True)
        self.flat = nn.Flatten()
        self.dense = nn.Linear(32, 1)

    def forward(self, x):
       out1, out2 = self.gru1(x)
       out1, out2 = self.gru2(out1)
       out = self.flat(out2.squeeze())
       out = self.dense(out)

       return out

In [24]:
rnn1 = RNN1()


In [27]:
class CNN1(nn.Module):
    def __init__(self):
        super(CNN1, self).__init__()
        self.conv1 = nn.Conv1d(64, 128, 7)
        self.max_polling = nn.MaxPool1d(5)
        self.conv1 = nn.Conv1d(128, 32, 7)
        self.max_polling = nn.MaxPool1d(5)
        self.gru1 = nn.GRU(4, 32, batch_first=True, bidirectional= False , num_layers = 1)
        self.gru2 = nn.GRU(32, 32, batch_first = True)
        self.flat = nn.Flatten()
        self.dense = nn.Linear(32, 1)
    def forward(self, x):
        x = x.permute(0, 2, 1)
        out = self.conv1(x)
        out = self.max_polling(out)
        out = self.conv2(out)
        out = self.max_polling(out)
        x = x.permute(0, 2, 1)
        
        out1, out2 = self.gru1(out)
        out1, out2 = self.gru2(out1)
        out = self.flat(out2.squeeze())
        out = self.dense(out)

        return out


In [ ]:
steps_per_epoch = 40

for idx, data in enumerate(Train_loader):
    if idx == steps_per_epoch:
        break